In [1]:
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()
import MySQLdb

def connection():
    conn = MySQLdb.connect(host="localhost",
                           user = "root",
                           passwd = "",
                           port=3306,
                           db = "ecommerce")
    c = conn.cursor()

    return c, conn	
c, conn=connection()
query = "SELECT user_id,age,review,verified,engagement FROM userdata"
df = pd.read_sql(query, conn)
data=df.values
userdata_dict = {row[0]: list(row[1:]) for row in data}
query = "SELECT user_id,frequency,similarity FROM userdata"
df = pd.read_sql(query, conn)
data=df.values
usual_dict = {row[0]: list(row[1:]) for row in data}
# Close connection
conn.close()

df

C:\Users\dell\AppData\Local\Temp\ipykernel_10672\907002996.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\dell\AppData\Local\Temp\ipykernel_10672\907002996.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,user_id,frequency,similarity
0,7c0d54b9-887c-42be-b,12,23
1,b1d1b7ca-56ab-40ff-a,4,5
2,eab3a7b6-5bac-44e1-b,33,44


In [2]:
c, conn=connection()
query = "SELECT product_id,category FROM product"
df = pd.read_sql(query, conn)
df_dict = df.set_index('product_id')['category'].to_dict()

C:\Users\dell\AppData\Local\Temp\ipykernel_10672\13255173.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [4]:
c, conn=connection()
query = "SELECT user_id,username FROM userdata"
df = pd.read_sql(query, conn)
dfuser_dict = df.set_index('user_id')['username'].to_dict()

C:\Users\dell\AppData\Local\Temp\ipykernel_10672\1532930326.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [5]:
dfuser_dict

{'7c0d54b9-887c-42be-b': 'mano',
 'b1d1b7ca-56ab-40ff-a': 'Peter',
 'eab3a7b6-5bac-44e1-b': 'Adhi'}

In [6]:

import pandas as pd

# Define the data
factors = ["Account Age", "Review History", "Verified Purchase", "Review Engagement"]
weights = [20, 15, 15, 10]  # In percentage
users = userdata_dict

# Compute weighted scores for each user
def compute_weighted_score(user_data, weights):
    data= sum(value * weight / 100 for value, weight in zip(user_data, weights))
    if(data>10):
        return 10
    else:
        return data

user_scores = {user: compute_weighted_score(data, weights) for user, data in users.items()}

# Unusual Pattern Score Factors
unusual_factors = ["Review Frequency", "Similarity in Content"]
unusual_weights = [20, 20]
unusual_users =usual_dict

# Compute unusual pattern scores
unusual_scores = {user: compute_weighted_score(data, unusual_weights) for user, data in unusual_users.items()}

# Compute Final Fake Review Score
final_scores = {
    user: (0.6 * user_scores[user]) + (0.4 * unusual_scores[user])
    for user in users
}

# Combine data
factors += unusual_factors + ["Total (Weighted 100%)", "Final Fake Review Score"]
weights += unusual_weights + [100, "-"]

for user in users:
    users[user] += unusual_users[user] + [unusual_scores[user], final_scores[user]]

data = {
    "Factor": factors,
    "Weight (%)": weights,
    **users
}

# Create DataFrame
df = pd.DataFrame(data)
df

,Factor,Weight (%),7c0d54b9-887c-42be-b,b1d1b7ca-56ab-40ff-a,eab3a7b6-5bac-44e1-b
0,Account Age,20,20.0,20.0,20.0
1,Review History,15,10.0,12.0,12.0
2,Verified Purchase,15,20.0,2.0,34.0
3,Review Engagement,10,20.0,2.0,12.0
4,Review Frequency,20,12.0,4.0,33.0
5,Similarity in Content,20,23.0,5.0,44.0
6,Total (Weighted 100%),100,7.0,1.8,10.0
7,Final Fake Review Score,-,8.8,4.5,10.0


In [7]:
user_scores

{'7c0d54b9-887c-42be-b': 10,
 'b1d1b7ca-56ab-40ff-a': 6.3,
 'eab3a7b6-5bac-44e1-b': 10}

In [8]:
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
# Download VADER lexicon (only once)
nltk.download('vader_lexicon')
# Initialize Sentiment Analyzer
sia = SentimentIntensityAnalyzer()
def get_sentiment_score(text):
    return sia.polarity_scores(text)["compound"]
c, conn=connection()
query = "SELECT * FROM reviewdata"
product = pd.read_sql(query, conn)
product["credibility_score"] = product["userid"].map(user_scores)
product["fake_review_score"] = product["userid"].map(final_scores)
product["UserName"] = product["userid"].map(dfuser_dict)


# Apply sentiment score to DataFrame
product["sentiment_score"] = product["review"].apply(get_sentiment_score)
product

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dell\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\dell\AppData\Local\Temp\ipykernel_10672\2723950466.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  product = pd.read_sql(query, conn)


,review_id,userid,productid,review,rating,frating,credibility_score,fake_review_score,UserName,sentiment_score
0,08403d44-813a-4202-8,7c0d54b9-887c-42be-b,10012,Very good and nice product,5,4,10.0,8.8,mano,0.7397
1,14b3a45b-63ea-4931-b,7c0d54b9-887c-42be-b,10000,nice produt,3,4,10.0,8.8,mano,0.4215
2,14b3a45b-63ea-4931-c,7c0d54b9-887c-42be-b,10001,nice produt,3,4,10.0,8.8,mano,0.4215
3,151284fb-cd8e-4e63-8,eab3a7b6-5bac-44e1-b,10000,Very good and nice product,3,4,10.0,10.0,Adhi,0.7397
4,54c057ce-0202-4eab-8,7c0d54b9-887c-42be-b,10022,Very good and nice product,2,4,10.0,8.8,mano,0.7397
5,57b853c6-0a69-416b-b,7c0d54b9-887c-42be-b,10002,Very good and nice product,4,4,10.0,8.8,mano,0.7397
6,663eb661-2cef-42b6-b,7c0d54b9-887c-42be-b,10010,Very good and nice product,5,4,10.0,8.8,mano,0.7397
7,7369aa27-7417-47a4-8,7c0d54b9-887c-42be-b,10002,Very good and nice product,4,4,10.0,8.8,mano,0.7397
8,a1d30e21-025e-47aa-a,7c0d54b9-887c-42be-b,2d93a3c5-1816-4b10-8,Very good and nice product,5,4,10.0,8.8,mano,0.7397
9,a44f1ad1-a31d-45b2-a,7c0d54b9-887c-42be-b,2d93a3c5-1816-4b10-8,Very good and nice product,4,4,10.0,8.8,mano,0.7397


In [9]:
grouped_df = product.groupby("productid").agg({
    "rating": "mean",
    "frating": "mean",
    "credibility_score": "mean",
    "fake_review_score": "mean",
    "sentiment_score": "mean"
}).reset_index()
product_df = grouped_df.to_dict(orient='records')
grouped_df

,productid,rating,frating,credibility_score,fake_review_score,sentiment_score
0,10000,3.000000,4.0,10.0,9.4,0.580600
1,10001,3.000000,4.0,10.0,8.8,0.421500
2,10002,4.000000,4.0,10.0,8.8,0.739700
3,10010,5.000000,4.0,10.0,8.8,0.739700
4,10012,3.666667,4.0,10.0,9.2,0.273467
5,10020,4.000000,4.0,10.0,8.8,0.739700
6,10022,2.000000,4.0,10.0,8.8,0.739700
7,2d93a3c5-1816-4b10-8,4.500000,4.0,10.0,8.8,0.739700
8,845a6112-485c-41ba-9,5.000000,4.0,10.0,8.8,0.739700


In [10]:
import pandas as pd

products =product_df

# Function to calculate Final Product Score
def calculate_final_score(credibility, sentiment, rating, fake_review,featurerating):
    # Calculate the Combined Product Score
    combined_score = (0.5 * credibility) + (0.3 * sentiment) + (0.1 * rating)+(0.1 * featurerating)
    # Calculate the Final Product Score
    final_score = combined_score * (1 - fake_review/10)
    return final_score

# Calculate final scores for all products and store them in the list
for product in products:
    final_score = calculate_final_score(
        product['credibility_score'],
        product['sentiment_score'],
        product['rating'],
        product['fake_review_score'],
        product['frating']
    )
    product['final_score'] = final_score

# Create a DataFrame from the product data
df = pd.DataFrame(products)
df["product_cata"] = df["productid"].map(df_dict)
# Sort the DataFrame by the final score in descending order
df['rank'] = df.groupby('product_cata')['final_score'].rank(ascending=False, method='dense')

# Sort for better readability
df = df.sort_values(by=['product_cata', 'rank'])
df

,productid,rating,frating,credibility_score,fake_review_score,sentiment_score,final_score,product_cata,rank
7,2d93a3c5-1816-4b10-8,4.500000,4.0,10.0,8.8,0.739700,0.728629,C001,1.0
2,10002,4.000000,4.0,10.0,8.8,0.739700,0.722629,C001,2.0
1,10001,3.000000,4.0,10.0,8.8,0.421500,0.699174,C001,3.0
0,10000,3.000000,4.0,10.0,9.4,0.580600,0.352451,C001,4.0
3,10010,5.000000,4.0,10.0,8.8,0.739700,0.734629,C002,1.0
8,845a6112-485c-41ba-9,5.000000,4.0,10.0,8.8,0.739700,0.734629,C002,1.0
4,10012,3.666667,4.0,10.0,9.2,0.273467,0.467897,C002,2.0
5,10020,4.000000,4.0,10.0,8.8,0.739700,0.722629,C003,1.0
6,10022,2.000000,4.0,10.0,8.8,0.739700,0.698629,C003,2.0


In [11]:
dfproduct_rate_dict = df.set_index('productid')['rank'].to_dict()

In [30]:
c, conn=connection()
query = "SELECT * FROM product"
df_product = pd.read_sql(query, conn)
df_product["rank"] = df_product["product_id"].map(dfproduct_rate_dict)


C:\Users\dell\AppData\Local\Temp\ipykernel_10672\1492729933.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_product = pd.read_sql(query, conn)


In [32]:
df_product

,product_id,quantity_pu,product_name,product_image,price,product_description,unit_weight,supplier,category,rank
0,10000,8,Mouse 2.0,mouse.JPG,400,A good camera fella,20,ABC123,C001,4.0
1,10001,12,Camera 1466FGFG,camera.JPG,15000,the OG camera,400,ABC123,C001,3.0
2,10002,10,Small adapters 2.0,smalladapters.JPG,800,adapters for everything,50,ABC123,C001,2.0
3,10010,10,Blue shirt,blueshirt.JPG,800,A good shirt mister,5,ABC123,C002,1.0
4,10012,10,Black Pants,blackpant.JPG,4000,the OG pant is back,10,ABC123,C002,2.0
5,10020,10,Car,car.JPEG,40,Toy car,5,ABC123,C003,1.0
6,10022,7,Teddy bear,bear.JPEG,200,the OG BEAR,10,ABC123,C003,2.0
7,2d93a3c5-1816-4b10-8,67,Headphone,3.jpg,23,fsdg fdgtfdyht,2,None,C001,1.0
8,845a6112-485c-41ba-9,3,T-Shirts,1.jpg,100,Otto Brand,200,None,C002,1.0
